In [1]:
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
%config IPCompleter.greedy=True

In [6]:
data_master = pd.read_fwf('STROKE_STRICT_ALL_DXS.txt',header=None)
cad_df = pd.read_excel('ICD_CAD_VHD.xlsx',sheet_name='IHD_CAD')
vhd_df = pd.read_excel('ICD_CAD_VHD.xlsx',sheet_name='VHD_NONRHEUM')


In [9]:
stroke_df = pd.read_excel('STROKE_STRICT_PROBLEM_LIST.xlsx', sheet_name='PROBLEM_LIST')
admission_df = pd.read_excel('STROKE_STRICT_PROBLEM_LIST.xlsx', sheet_name='ADMISION_ENCS')
geisinger_df = pd.read_excel('GNSIS_DATABASE_v7.5_6.25.2020.xlsx')

In [10]:
data_master.columns = ['PT_ID','Dates','ENC_TYPE','ICD_CD','ICD_NM','CODE_SYSTEM','PRIMARY_YN'] 
data_master = data_master.iloc[2:]
data_master = data_master.iloc[:-3]
data_master[['INDEX_DT','ENC_DT']] = data_master['Dates'].str.split(expand=True) 
del data_master['Dates']
print(data_master['PT_ID'].nunique())

9243


In [11]:
stroke_df=stroke_df.rename(columns={'PROB_DT':'ENC_DT'})
stroke_df['ENC_TYPE'] = 'IP ONLY'
print(stroke_df['PT_ID'].nunique())
print(admission_df['PT_ID'].nunique())

8917
16


In [12]:
data_master = data_master[['PT_ID','INDEX_DT','ENC_DT','ENC_TYPE','ICD_CD','ICD_NM']]
admission_df = admission_df[['PT_ID','INDEX_DT','ENC_DT','ENC_TYPE','ICD_CD','ICD_NM']]

In [13]:
data = data_master.append(stroke_df, ignore_index = True,sort=True) 
data = data.append(admission_df, ignore_index = True,sort=True) 

In [14]:
print(data['PT_ID'].nunique())
df = pd.merge(geisinger_df['PT_ID'], data, on=['PT_ID'], how='inner')
print(df['PT_ID'].nunique())
print(geisinger_df['PT_ID'].nunique())

9243
8929
8929


In [16]:
df_cad = df[df['ICD_CD'].isin (cad_df.ICD_CODE)]
df_vhd = df[df['ICD_CD'].isin (vhd_df.ICD_CODE)]
print(df_cad['PT_ID'].nunique())
print(df_vhd['PT_ID'].nunique())

3867
1679


# Using rule of 2 (1 inpatient, 2 out patient contribute to 1)

In [20]:
df_cad_outPatient = df_cad[df_cad['ENC_TYPE'] == 'OUTPATIENT'].copy()
df_cad_inPatient = df_cad[df_cad['ENC_TYPE'] != 'OUTPATIENT']
print('CAD Only')
print('outPatient: ',df_cad_outPatient['PT_ID'].nunique())
print('inPatient: ',df_cad_inPatient['PT_ID'].nunique())

print('===================================================')

df_vhd_outPatient = df_vhd[df_vhd['ENC_TYPE'] == 'OUTPATIENT'].copy()
df_vhd_inPatient = df_vhd[df_vhd['ENC_TYPE'] != 'OUTPATIENT']
print('VHD only')
print('outPatient: ',df_vhd_outPatient['PT_ID'].nunique())
print('inPatient: ',df_vhd_inPatient['PT_ID'].nunique())

df_vhd_outPatient.dtypes

CAD Only
outPatient:  2378
inPatient:  3680
VHD only
outPatient:  1093
inPatient:  1509


PT_ID       object
ENC_DT      object
ENC_TYPE    object
ICD_CD      object
ICD_NM      object
INDEX_DT    object
dtype: object

In [21]:
df_cad_outPatient = df_cad_outPatient.groupby('PT_ID')
df_cad_outPatient = df_cad_outPatient.filter(lambda x: len(x) > 1)

print(df_cad_outPatient['PT_ID'].nunique())
df_cad_outPatient=df_cad_outPatient.sort_values(by=['PT_ID','ENC_DT'])
testVal= df_cad_outPatient.groupby('PT_ID',as_index=False)['ENC_DT'].apply(lambda x: x.sort_values())
df_cad_outPatient['ENC_DT'] = testVal.reset_index(level=0, drop=True)


1985


In [ ]:
df_cad_outPatient

In [ ]:
df_cad_outPatient = df_cad_outPatient.groupby('PT_ID').apply(lambda x: x.tail(len(x)-1))\
        .reset_index(drop=True)

In [31]:
df_cad_inPatient['INDEX_DT']=pd.to_datetime(df_cad_inPatient['INDEX_DT'])
df_cad_inPatient['ENC_DT']=pd.to_datetime(df_cad_inPatient['ENC_DT'])
cad_inPatient_at_index = df_cad_inPatient[df_cad_inPatient['INDEX_DT']>= df_cad_inPatient['ENC_DT']].copy()

df_cad_outPatient['INDEX_DT']=pd.to_datetime(df_cad_outPatient['INDEX_DT'])
df_cad_outPatient['ENC_DT']=pd.to_datetime(df_cad_outPatient['ENC_DT'])

cad_outPatient_at_index = df_cad_outPatient[df_cad_outPatient['INDEX_DT']>= df_cad_outPatient['ENC_DT']].copy()
cad_patient_at_index = cad_inPatient_at_index.append(cad_outPatient_at_index)
print(cad_inPatient_at_index['PT_ID'].nunique())
print(cad_outPatient_at_index['PT_ID'].nunique())
print(cad_patient_at_index['PT_ID'].nunique())

2662
1345
2718


c:\users\mobas\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\mobas\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [32]:
df_vhd_outPatient = df_vhd_outPatient.groupby('PT_ID')
df_vhd_outPatient = df_vhd_outPatient.filter(lambda x: len(x) > 1)

print(df_vhd_outPatient['PT_ID'].nunique())
df_vhd_outPatient=df_vhd_outPatient.sort_values(by=['PT_ID','ENC_DT'])
testVal= df_vhd_outPatient.groupby('PT_ID',as_index=False)['ENC_DT'].apply(lambda x: x.sort_values())
df_vhd_outPatient['ENC_DT'] = testVal.reset_index(level=0, drop=True)

df_vhd_outPatient = df_vhd_outPatient.groupby('PT_ID').apply(lambda x: x.tail(len(x)-1))\
        .reset_index(drop=True)

843


In [33]:
df_vhd_inPatient['INDEX_DT']=pd.to_datetime(df_vhd_inPatient['INDEX_DT'])
df_vhd_inPatient['ENC_DT']=pd.to_datetime(df_vhd_inPatient['ENC_DT'])
vhd_inPatient_at_index = df_vhd_inPatient[df_vhd_inPatient['INDEX_DT']>= df_vhd_inPatient['ENC_DT']].copy()

df_vhd_outPatient['INDEX_DT']=pd.to_datetime(df_vhd_outPatient['INDEX_DT'])
df_vhd_outPatient['ENC_DT']=pd.to_datetime(df_vhd_outPatient['ENC_DT'])

vhd_outPatient_at_index = df_vhd_outPatient[df_vhd_outPatient['INDEX_DT']>= df_vhd_outPatient['ENC_DT']].copy()

vhd_patient_at_index = vhd_inPatient_at_index.append(vhd_outPatient_at_index)
print(vhd_inPatient_at_index['PT_ID'].nunique())
print(vhd_outPatient_at_index['PT_ID'].nunique())
print(vhd_patient_at_index['PT_ID'].nunique())

966
457
1007


c:\users\mobas\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\mobas\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [45]:
geisinger_afib_panda = pd.read_csv('geisinger_afib_patients.csv',low_memory=False)
geisinger_non_afib_panda = pd.read_csv('geisinger_non_afib_patients.csv',low_memory=False)

geisinger_afib_aflt_panda = pd.read_csv('geisinger_afib_aflt_patients.csv',low_memory=False)
geisinger_non_afib_aflt_panda = pd.read_csv('geisinger_non_afib_aflt_patients.csv',low_memory=False)

In [56]:
geisinger_afib_panda['CAD_AT_INDEX'] = 0
geisinger_afib_panda.loc[geisinger_afib_panda['PT_ID'].isin(cad_patient_at_index['PT_ID'].unique()), 'CAD_AT_INDEX'] = 1

geisinger_afib_panda['VHD_AT_INDEX'] = 0
geisinger_afib_panda.loc[geisinger_afib_panda['PT_ID'].isin(vhd_patient_at_index['PT_ID'].unique()), 'VHD_AT_INDEX'] = 1

geisinger_non_afib_panda['CAD_AT_INDEX'] = 0
geisinger_non_afib_panda.loc[geisinger_non_afib_panda['PT_ID'].isin(cad_patient_at_index['PT_ID'].unique()), 'CAD_AT_INDEX'] = 1

geisinger_non_afib_panda['VHD_AT_INDEX'] = 0
geisinger_non_afib_panda.loc[geisinger_non_afib_panda['PT_ID'].isin(vhd_patient_at_index['PT_ID'].unique()), 'VHD_AT_INDEX'] = 1


geisinger_afib_aflt_panda['CAD_AT_INDEX'] = 0
geisinger_afib_aflt_panda.loc[geisinger_afib_aflt_panda['PT_ID'].isin(cad_patient_at_index['PT_ID'].unique()), 'CAD_AT_INDEX'] = 1

geisinger_afib_aflt_panda['VHD_AT_INDEX'] = 0
geisinger_afib_aflt_panda.loc[geisinger_afib_aflt_panda['PT_ID'].isin(vhd_patient_at_index['PT_ID'].unique()), 'VHD_AT_INDEX'] = 1

geisinger_non_afib_aflt_panda['CAD_AT_INDEX'] = 0
geisinger_non_afib_aflt_panda.loc[geisinger_non_afib_aflt_panda['PT_ID'].isin(cad_patient_at_index['PT_ID'].unique()), 'CAD_AT_INDEX'] = 1

geisinger_non_afib_aflt_panda['VHD_AT_INDEX'] = 0
geisinger_non_afib_aflt_panda.loc[geisinger_non_afib_aflt_panda['PT_ID'].isin(vhd_patient_at_index['PT_ID'].unique()), 'VHD_AT_INDEX'] = 1

In [92]:
geisinger_afib_panda[geisinger_afib_panda['OBESITY_AT_INDEX']==1]['PT_ID'].nunique()

823

In [103]:
geisinger_afib_aflt_panda[geisinger_afib_aflt_panda['OBESITY_AT_INDEX']==1]['PT_ID'].nunique()

841

In [104]:
geisinger_non_afib_aflt_panda[geisinger_non_afib_aflt_panda['OBESITY_AT_INDEX']==1]['PT_ID'].nunique()

1866

In [63]:
geisinger_non_afib_aflt_panda[geisinger_non_afib_aflt_panda['VHD_AT_INDEX']==1]['PT_ID'].nunique()
geisinger_df.dtypes

PT_ID                                                        object
PT_SEX                                                       object
PT_BIRTH_DT                                          datetime64[ns]
PT_DEATH_DT                                          datetime64[ns]
LAST_ACTIVE_DT                                       datetime64[ns]
                                                          ...      
CREATININE_MEDIAN_AFTER_INDEX                               float64
CREATININE_CLOSEST_BEFORE_RECUR_OR_LASTENC                  float64
DT_CREATININE_CLOSEST_BEFORE_RECUR_OR_LASTENC        datetime64[ns]
CREATININE_CLOSEST_BEFORE_DEATH_OR_LASTENC                  float64
DT_FOR_CREATININE_CLOSEST_BEFORE_DEATH_OR_LASTENC    datetime64[ns]
Length: 284, dtype: object

In [64]:
geisinger_afib_panda.to_csv('geisinger_afib_patients.csv',index=False)
geisinger_non_afib_panda.to_csv('geisinger_non_afib_patients.csv',index=False)

geisinger_afib_aflt_panda.to_csv('geisinger_afib_aflt_patients.csv',index=False)
geisinger_non_afib_aflt_panda.to_csv('geisinger_non_afib_aflt_patients.csv',index=False)


In [80]:
import numpy as np
geisinger_df['DT_FOR_BMI_CLOSEST_TO_INDEX']=pd.to_datetime(geisinger_df['DT_FOR_BMI_CLOSEST_TO_INDEX'])
geisinger_df['INDEX_DT']=pd.to_datetime(geisinger_df['INDEX_DT'])
geisinger_df['diff'] = (geisinger_df['INDEX_DT']-geisinger_df['DT_FOR_BMI_CLOSEST_TO_INDEX'])/np.timedelta64(1,'Y')

In [83]:
geisinger_df[geisinger_df['diff']>=0]['diff'].describe()

count   6966.000
mean       0.011
std        0.117
min        0.000
25%        0.000
50%        0.000
75%        0.000
max        2.913
Name: diff, dtype: float64

In [100]:
geisinger_obesity = geisinger_df[geisinger_df['DT_FOR_BMI_CLOSEST_TO_INDEX'] <= geisinger_df['INDEX_DT']]
geisinger_obesity_greate_30 = geisinger_obesity[geisinger_obesity['BMI_CLOSEST_TO_INDEX'] > 30]
geisinger_obesity_greate_30['PT_ID'].nunique()

2707

In [101]:
geisinger_afib_panda['OBESITY_AT_INDEX'] = 0
geisinger_afib_panda.loc[geisinger_afib_panda['PT_ID'].isin(geisinger_obesity_greate_30['PT_ID'].unique()), 'OBESITY_AT_INDEX'] = 1

geisinger_non_afib_panda['OBESITY_AT_INDEX'] = 0
geisinger_non_afib_panda.loc[geisinger_non_afib_panda['PT_ID'].isin(geisinger_obesity_greate_30['PT_ID'].unique()), 'OBESITY_AT_INDEX'] = 1

geisinger_afib_aflt_panda['OBESITY_AT_INDEX'] = 0
geisinger_afib_aflt_panda.loc[geisinger_afib_aflt_panda['PT_ID'].isin(geisinger_obesity_greate_30['PT_ID'].unique()), 'OBESITY_AT_INDEX'] = 1

geisinger_non_afib_aflt_panda['OBESITY_AT_INDEX'] = 0
geisinger_non_afib_aflt_panda.loc[geisinger_non_afib_aflt_panda['PT_ID'].isin(geisinger_obesity_greate_30['PT_ID'].unique()), 'OBESITY_AT_INDEX'] = 1

In [102]:
geisinger_afib_panda.dtypes

PT_ID                                                 object
PT_SEX                                                object
PT_BIRTH_DT                                           object
PT_DEATH_DT                                           object
LAST_ACTIVE_DT                                        object
                                                      ...   
CREATININE_CLOSEST_BEFORE_DEATH_OR_LASTENC           float64
DT_FOR_CREATININE_CLOSEST_BEFORE_DEATH_OR_LASTENC     object
CAD_AT_INDEX                                           int64
VHD_AT_INDEX                                           int64
OBESITY_AT_INDEX                                       int64
Length: 287, dtype: object

In [105]:
geisinger_afib_panda.to_csv('geisinger_afib_patients.csv',index=False)
geisinger_non_afib_panda.to_csv('geisinger_non_afib_patients.csv',index=False)

geisinger_afib_aflt_panda.to_csv('geisinger_afib_aflt_patients.csv',index=False)
geisinger_non_afib_aflt_panda.to_csv('geisinger_non_afib_aflt_patients.csv',index=False)